In [ ]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [ ]:
spark = SparkSession.builder.appName("ETL-local").getOrCreate()

In [ ]:
dataframe = spark.read.csv("./data/users.csv", header=True, inferSchema=True)
dataframe.show(5)

In [ ]:
dataframe_clean = dataframe.filter(
    dataframe.age.isNotNull()
).filter(
    dataframe.country != "Unknown"
)

In [ ]:
dataframe_country = dataframe_clean.groupBy("country").agg(
    {
        "age": "avg",
        "salary": "avg"
    }
)
dataframe_country.show()

In [ ]:
dataframe = dataframe.withColumn(
    "age_category",
    when(col("age") < 30, "jeune")
    .when(col("age") < 45, "adulte")
    .otherwise("senior")
)

In [ ]:
dataframe.groupBy("country", "age_category").agg(
    {
        "salary": "avg"
    }
).show()

In [ ]:
os.makedirs("./data/output", exist_ok=True)
dataframe_country.write.mode("overwrite").parquet("./data/output/country_stats.parquet")

In [ ]:
read_parquet = pd.read_parquet("./data/output/country_stats.parquet/", engine="pyarrow")
read_parquet

In [ ]:
spark.stop()